# 第六章 偏微分方程的数值方法 (未解决)

## 扩展题

### 6.2 二维不可压缩流体 Navier - Stokes 方程

### 1）方程描述。设
$$
\begin{align*}
\boldsymbol{q}&=(u, v),\\
\nabla\phi&=\left(\frac{\partial\phi}{\partial x}, \frac{\partial\phi}{\partial y}\right), \\
\Delta\phi&=\left(\frac{\partial^2\phi}{\partial x^2}, \frac{\partial^2\phi}{\partial y^2}\right), \\
-\boldsymbol{H}&=\left(\frac{\partial u^2}{\partial x}+\frac{\partial uv}{\partial y}, \frac{\partial uv}{\partial x}+\frac{\partial v^2}{\partial y}\right)
\end{align*}
$$

质量守恒方程
$$\frac{\partial u}{\partial x}+\frac{\partial v}{\partial y} = 0$$

即
$$\text{div}(\boldsymbol{q}) = 0$$

动量守恒方程
$$
\begin{cases} 
\displaystyle \frac{\partial u}{\partial t}+\frac{\partial u^2}{\partial x}+\frac{\partial uv}{\partial y}=-\frac{\partial p}{\partial x}+\frac{1}{Re}\left( \frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right) \\
\displaystyle \frac{\partial v}{\partial t}+\frac{\partial uv}{\partial x}+\frac{\partial v^2}{\partial y}=-\frac{\partial p}{\partial y}+\frac{1}{Re}\left( \frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2} \right) 
\end{cases}
$$

即  
$$\frac{\partial \boldsymbol{q}}{\partial t}=-\nabla p + \boldsymbol{H} + \frac{1}{Re}\Delta \boldsymbol{q}$$  


### （2）基本数值算法  
① 显式计算  
$$H_u^n = -\left( \frac{\partial u^2}{\partial x}+\frac{\partial uv}{\partial y} \right),\quad H_v^n = -\left( \frac{\partial uv}{\partial x}+\frac{\partial v^2}{\partial y} \right)$$  

② 由解 Helmholtz 方程计算非螺旋场量 $\boldsymbol{q}^*=(u^*,v^*)$，  
$$\left( I - \frac{\delta t}{2Re}\Delta \right)u^* = u^n + \delta t\left( -\frac{\partial p^n}{\partial x}+\frac{3}{2}H_u^n - \frac{1}{2}H_u^{n - 1}+\frac{1}{2Re}\Delta u^n \right)$$  
$$\left( I - \frac{\delta t}{2Re}\Delta \right)v^* = v^n + \delta t\left( -\frac{\partial p^n}{\partial y}+\frac{3}{2}H_v^n - \frac{1}{2}H_v^{n - 1}+\frac{1}{2Re}\Delta v^n \right)$$  

③ 由解泊松方程计算中间变量 $\Delta\phi = \frac{1}{\delta t}\left( \frac{\partial u^*}{\partial x}+\frac{\partial v^*}{\partial y} \right)$；  

④ 计算螺旋场 $\boldsymbol{q}^{n + 1}=(u^{n + 1},v^{n + 1})$，$u^{n + 1}=u^* - \delta t\frac{\partial \phi}{\partial x}$，$v^{n + 1}=v^* - \delta t\frac{\partial \phi}{\partial y}$；  

⑤ 计算新的压强 $p^{n + 1}=p^n + \phi - \frac{\delta t}{2Re}\Delta \phi$；  


在每个时间步 $\delta t$ 上重复①～⑤ 。

### （3）边界条件和 Staggered 网格选取  
取 $L_x \times L_y$ 矩形区域，边界条件为周期性速度场和压强场  
$$
\begin{cases} 
\boldsymbol{q}(0, y) = \boldsymbol{q}(L_x, y),\quad p(0, y) = p(L_x, y) \\
\boldsymbol{q}(x, 0) = \boldsymbol{q}(x, L_y),\quad p(x, 0) = p(x, L_y) 
\end{cases}
$$  

建立两套网格点  
$$\delta x = L_x/(n_x - 1),\quad \delta y = L_y/(n_y - 1)$$  

一套是建立在网胞的四个顶点上  
$$x_c(i) = (i - 1)\delta x,\ i = 1,\cdots,n_x;\quad y_c(j) = (j - 1)\delta y,\ j = 1,\cdots,n_y$$  

另一套是建立在网胞的中心  
$$x_m(i) = (i - 1/2)\delta x,\ i = 1,\cdots,n_x - 1$$  
$$y_m(j) = (j - 1/2)\delta y,\ j = 1,\cdots,n_y - 1$$  


而 $u, v, p$ 的计算点定义为  
- $u(i, j) \approx u(x_c(i), y_m(j))$，在 $(i, j)$ 胞的左边中点；  
- $v(i, j) \approx v(x_m(i), y_c(j))$，在 $(i, j)$ 胞的下底边中点；  
- $p(i, j) \approx p(x_m(i), y_m(j))$，在 $(i, j)$ 胞的中点。  


### （4）时间步长选取  
$$
\mathrm{d}t = \frac{cfl}{\max\left( \left| \frac{u}{\delta x} \right| + \left| \frac{v}{\delta y} \right| \right)},\quad cfl < 1
$$

### （5）编程数值模拟 Kelvin - Helmholtz 不稳定性
#### 初始的流场分布
$$v(x, y) = 0, \quad u(x, y) = u_1(y)(1 + u_2(x))$$  

- $u_1$ 是平均速度分布，取  
$$u_1(y) = \frac{U_0}{2}\left( 1 + \tanh \left( \frac{1}{2}P_j \left( 1 - \frac{|L_y/2 - y|}{R_j} \right) \right) \right)$$  

- $u_2$ 是触发 Kelvin - Helmholtz 不稳定性的扰动速度  
$$u_2(x) = A_x \sin(2\pi x / \lambda_x)$$  


#### 参数建议  
$$
\begin{align*}
L_x &= 2, & L_y &= 1, & n_x &= 65, & n_y &= 65, & cfl &= 0.2 \\
Re &= 1000, & U_0 &= 1, & P_j &= 20, & R_j &= L_y/4, & A_x &= 0.5, & \lambda_x &= 0.5L_x \\
\end{align*}
$$
可以改变某些参数，包括初始分布数值模拟体系的演化情况。
